# Molecular Dynamics: vinculin--alpha-catenin
### University of California, Berkeley - Spring 2024

---
## Molecular dynamics of proteins <a id='l_md'></a>

[**Molecular dynamics (MD)**](https://en.wikipedia.org/wiki/Molecular_dynamics) is a computer simulation method for studying the physical movements of atoms and molecules, i.e. their dynamical evolution. 

In the most common version, the trajectories of atoms and molecules are determined by numerically solving Newton's equations of motion for a system of interacting particles, where forces between the particles and their potential energies are often calculated using  molecular mechanics force fields. 



Now with all that intellectual equipment, we can start running legit Molecular Dynamics simulations. All we need is an initial structure of the protein and software that computes its dynamics efficiently.

### Procedure

1. Load initial coordinates of protein atoms (from *.pdb file)
2. Choose force field parameters (in potential function V from section 5).
3. Choose parameters of the experiment: temperature, pressure, box size, solvation, boundary conditions
4. Choose integrator, i.e. algorithm for solving equation of motion
5. Run simulation, save coordinates time to time (to *.dcd file).
6. Visualize the trajectory
7. Perform the analysis

__Packages__

* __NGLViewer__: NGL Viewer is a collection of tools for web-based molecular graphics. WebGL is employed to display molecules like proteins and DNA/RNA with a variety of representations.

* __MDAnalysis__: MDAnalysis is an object-oriented Python library to analyze trajectories from molecular dynamics (MD) simulations in many popular formats. It can write most of these formats, too, together with atom selections suitable for visualization or native analysis tools.

* __Openmm__: Openmm consists of two parts: One is a set of libraries that lets programmers easily add molecular simulation features to their programs and the other parts is an “application layer” that exposes those features to end users who just want to run simulations

---
### Example: MD simulation of 4EHP 

In [ ]:
from openmm.app import *
from openmm import *
from openmm.unit import *
import MDAnalysis as mda
import nglview as nv
from sys import stdout
import pathlib

In [ ]:
# Load PDB file
input = pathlib.Path(
    # Uses a local file
     "data/4ehp.pdb"
    # "data/alpha_catenin.pdb"
    # "data/vinculin.pdb"
    

)
protein = input.name.split(".")[0]

pdb = PDBFile(str(input))

import MDAnalysis as mda
from MDAnalysis.topology import guessers

# Load your PDB file
u = mda.Universe('results/4ehp_300K_traj.pdb')

# Guess the elements
u.atoms.elements = guessers.guess_types(u.atoms.names)


In [ ]:
# Create system
forcefield = ForceField(
    # 'amber99sb.xml',
    # 'tip3p.xml',

    # 'amber10.xml',

    'amber14-all.xml',
    'amber14/tip3pfb.xml'
    # HINT: Use the amber14 force fields for villin
)
# Extra steps to clean up the system:
modeller = Modeller(pdb.topology, pdb.positions)
modeller.deleteWater()
residues = modeller.addHydrogens(forcefield)
modeller.addSolvent(forcefield, padding=1.0*nanometer)

system = forcefield.createSystem(
    pdb.topology,
    nonbondedMethod=NoCutoff,
    constraints=HBonds
)

# Set parameters for the experiment
temperature = 300*kelvin
friction = 1/picosecond
timestep = 0.002*picoseconds
total_steps = 400*picoseconds/timestep
traj_record_freq = 1000  # Save trajectory every 1000 steps, decrease to show more detail in simualtion

# Set up integrator
integrator = LangevinIntegrator(
    temperature,
    friction,
    timestep
)

# Set up simulation
simulation = Simulation(pdb.topology, system, integrator)
simulation.context.setPositions(pdb.positions)

# Minimize energy
simulation.minimizeEnergy()
print(f"Running dynamics for {total_steps} steps on {protein}, saving trajectory every {traj_record_freq} steps...")
# Add reporter to save trajectory
simulation.reporters.append(PDBReporter(f"results/{protein}_{str(temperature)[:3]}K_traj.pdb", traj_record_freq))
# Print progress to standard output
simulation.reporters.append(StateDataReporter(
    stdout,
    total_steps//10,
    step=True,
    potentialEnergy=True,
    temperature=True,
    progress=True,
    totalSteps = total_steps
))

# Run simulation
simulation.step(total_steps)

traj = pathlib.Path(f"results/{protein}_{str(temperature)[:3]}K_traj.pdb")
print(f"Simulation Complete.\nTrajectory saved to {traj}")
# Load trajectory using MDAnalysis
u = mda.Universe(input, traj)

In [ ]:
# Visualize trajectory using nglview
view = nv.show_mdanalysis(u, show_gui=True)
view